In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
calendar = pd.DataFrame({
    'Week Range': pd.date_range(start='2022-01-01', end=dt.datetime.now()).to_period('W').unique()
})

calendar[['Week Start', 'Week End']] = calendar['Week Range'].astype(str).str.split('/', expand=True)
calendar['Week Start'] = pd.to_datetime(calendar['Week Start'])
calendar.reset_index(inplace=True)
calendar.rename(columns={'index':'Week Number'}, inplace=True)
calendar['Week Number'] = calendar['Week Number'] + 1
calendar = calendar[['Week Range', 'Week Start', 'Week End', 'Week Number']]
calendar.sort_values(by='Week Start', ascending=False, inplace=True)
calendar.reset_index(drop=True, inplace=True)
calendar

,Week Range,Week Start,Week End,Week Number
0,2022-03-14/2022-03-20,2022-03-14,2022-03-20,12
1,2022-03-07/2022-03-13,2022-03-07,2022-03-13,11
2,2022-02-28/2022-03-06,2022-02-28,2022-03-06,10
3,2022-02-21/2022-02-27,2022-02-21,2022-02-27,9
4,2022-02-14/2022-02-20,2022-02-14,2022-02-20,8
5,2022-02-07/2022-02-13,2022-02-07,2022-02-13,7
6,2022-01-31/2022-02-06,2022-01-31,2022-02-06,6
7,2022-01-24/2022-01-30,2022-01-24,2022-01-30,5
8,2022-01-17/2022-01-23,2022-01-17,2022-01-23,4
9,2022-01-10/2022-01-16,2022-01-10,2022-01-16,3


In [3]:
df = pd.DataFrame({
    'Date Pulled': ['2022-01-25', '2022-01-17', '2022-01-10'],
    'Column': ['Test', 'Test', 'Test']
})

df['Date Pulled'] = pd.to_datetime(df['Date Pulled'])
df

,Date Pulled,Column
0,2022-01-25,Test
1,2022-01-17,Test
2,2022-01-10,Test


In [4]:
df_merge = pd.merge(df, calendar, how='outer', left_on='Date Pulled', right_on='Week Start')
df_merge.sort_values(by='Week Number', ascending=False, inplace=True)
df_merge.reset_index(drop=True, inplace=True)
df_merge

,Date Pulled,Column,Week Range,Week Start,Week End,Week Number
0,NaT,NaN,2022-03-14/2022-03-20,2022-03-14,2022-03-20,12.0
1,NaT,NaN,2022-03-07/2022-03-13,2022-03-07,2022-03-13,11.0
2,NaT,NaN,2022-02-28/2022-03-06,2022-02-28,2022-03-06,10.0
3,NaT,NaN,2022-02-21/2022-02-27,2022-02-21,2022-02-27,9.0
4,NaT,NaN,2022-02-14/2022-02-20,2022-02-14,2022-02-20,8.0
5,NaT,NaN,2022-02-07/2022-02-13,2022-02-07,2022-02-13,7.0
6,NaT,NaN,2022-01-31/2022-02-06,2022-01-31,2022-02-06,6.0
7,NaT,NaN,2022-01-24/2022-01-30,2022-01-24,2022-01-30,5.0
8,2022-01-17,Test,2022-01-17/2022-01-23,2022-01-17,2022-01-23,4.0
9,2022-01-10,Test,2022-01-10/2022-01-16,2022-01-10,2022-01-16,3.0


In [5]:
test_null = df_merge[df_merge['Week Number'].isnull()]
test_null

,Date Pulled,Column,Week Range,Week Start,Week End,Week Number
12,2022-01-25,Test,NaT,NaT,NaN,NaN


In [6]:
if test_null['Date Pulled'][test_null['Date Pulled'] == '2022-01-25'].item() == pd.to_datetime('2022-01-25'):
    print('nice')
else:
    print('not nice')

nice


In [7]:
date_list = df_merge['Week Start'].tolist()

[Timestamp('2022-03-14 00:00:00'),
 Timestamp('2022-03-07 00:00:00'),
 Timestamp('2022-02-28 00:00:00'),
 Timestamp('2022-02-21 00:00:00'),
 Timestamp('2022-02-14 00:00:00'),
 Timestamp('2022-02-07 00:00:00'),
 Timestamp('2022-01-31 00:00:00'),
 Timestamp('2022-01-24 00:00:00'),
 Timestamp('2022-01-17 00:00:00'),
 Timestamp('2022-01-10 00:00:00'),
 Timestamp('2022-01-03 00:00:00'),
 Timestamp('2021-12-27 00:00:00'),
 NaT]

In [8]:
# REFERENCE CODE - Subset example

# # Filters main dataframe if the job title contains all the job terms that was inputted by the user
# job_terms_subset = set(job_terms).issubset
# df = df[[job_terms_subset(l) for l in df['job_title'].str.split()]]
# display(df.head())
# print('Rows, columns:', df.shape)

In [13]:
date_set = set(df_merge['Date Pulled']).issubset
df_test = df_merge[[date_set(i) for i in df_merge['Week Start']]]

TypeError: 'Timestamp' object is not iterable